<a href="https://colab.research.google.com/github/joheras/MusicaCatedralStoDomingoIER/blob/main/Notebooks/SacarM%C3%BAsica/image2xml.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# input.jpg $\rightarrow$ output.xml
### Cuaderon para transformar las predicciones al formato musicxml

## Librerías

In [1]:
%%capture
pip install py_midicsv

In [2]:
%%capture
pip install MIDIUtil

In [3]:
%%capture
pip install pretty_midi

In [4]:
%%capture
!pip install icevision[all]==0.5.1

In [5]:
%%capture
!pip install torchtext==0.8.1

In [6]:
%%capture
!pip install fastai --upgrade

In [7]:
%%capture
!sudo apt install tesseract-ocr
!sudo apt-get install tesseract-ocr-spa
!pip install pytesseract
!pip install tesseract

In [1]:
from icevision.all import *
from fastai.vision.all import *
import cv2
import pickle
import midiutil
import pretty_midi
from pytesseract import Output
import pytesseract
import IPython.display as ipd
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from matplotlib import patches
from matplotlib import colors
import statistics
import py_midicsv as pm
import random
import sys
import os
import json
import music21 as m21

## Dataset y funciones

In [ ]:
%%capture
!unzip dataset.zip

In [3]:
def get_iou(bb1, bb2):
    assert bb1[0] < bb1[2]
    assert bb1[1] < bb1[3]
    assert bb2[0] < bb2[2]
    assert bb2[1] < bb2[3]
    x_left = max(bb1[0], bb2[0])
    y_top = max(bb1[1], bb2[1])
    x_right = min(bb1[2], bb2[2])
    y_bottom = min(bb1[3], bb2[3])
    if x_right < x_left or y_bottom < y_top:
        return 0.0
    intersection_area = (x_right - x_left) * (y_bottom - y_top)
    bb1_area = (bb1[2] - bb1[0]) * (bb1[3] - bb1[1])
    bb2_area = (bb2[2] - bb2[0]) * (bb2[3] - bb2[1])
    iou = intersection_area / float(bb1_area + bb2_area - intersection_area)
    assert iou >= 0.0
    assert iou <= 1.0
    return iou

In [4]:
d={'background':0,'claved':1, 'claves':2, 'clavef':3, 'nota1':4, 'nota2':5, 'nota4':6, 'nota1:2':7, 'nota1:4':8, 'notae':9, 'sos':10, 'bem':11, 'bec':12, 'sil0':13, 'sil1':14, 'sil2':15, 'sil1:2':16, 'sil1:4':17, '2x4':18, '3x4':19, '4x4':20, '6x8':21, 'punt':22, 'liga':23,'3':24,'line':25,'P2':26, 'nota1:8':27, 'cal':28, '3x2':29,'mf':30,'P1':31,'check':32,'3x8':33,'2x2':34}
d = {v: k for k, v in d.items()}

def qsort(bboxs,labels,scores):
  if bboxs == []:
    return bboxs,labels,scores
  bbox = bboxs[0]
  label = labels[0]
  score = scores[0]
  n = len(labels[1:])
  bboxsL,labelsL,scoresL = qsort([b for b in bboxs[1:] if b[0] < bbox[0]],[labels[1:][i] for i in range(0,n) if bboxs[1:][i][0] < bbox[0]],[scores[1:][i] for i in range(0,n) if bboxs[1:][i][0] < bbox[0]])
  bboxsU,labelsU,scoresU = qsort([b for b in bboxs[1:] if b[0] >= bbox[0]],[labels[1:][i] for i in range(0,n) if bboxs[1:][i][0] >= bbox[0]],[scores[1:][i] for i in range(0,n) if bboxs[1:][i][0] >= bbox[0]])

  return bboxsL + [bbox] + bboxsU, labelsL + [label] + labelsU, scoresL + [score] + scoresU

def image2pred(path):
  test = cv2.imread(path)
  Y,X,_=test.shape
  infer_tfms = tfms.A.Adapter([*tfms.A.resize_and_pad(size=923), tfms.A.Normalize()])
  infer_ds = Dataset.from_images([test], infer_tfms)
  infer_dl = faster_rcnn.infer_dl(infer_ds, batch_size=1)
  samples, preds = faster_rcnn.predict_dl(model=model, infer_dl=infer_dl)
  L = []
  labels = list(preds[0]['labels'])
  bboxs = preds[0]['bboxes']
  for i in range(0,len(labels)):
    xmin=bboxs[i].xmin
    ymin=bboxs[i].ymin-(samples[0]['img'].shape[0]-samples[0]['height'])/2
    xmax=bboxs[i].xmax
    ymax=bboxs[i].ymax-(samples[0]['img'].shape[0]-samples[0]['height'])/2
    xmin=int(X*xmin/samples[0]['width'])
    ymin=int(Y*ymin/samples[0]['height'])
    xmax=int(X*xmax/samples[0]['width'])
    ymax=int(Y*ymax/samples[0]['height'])
    L.append([xmin,ymin,xmax,ymax])
  scores = list(preds[0]['scores'])
  L,labels,scores = qsort(L,labels,scores)
  labels = [d[l] for l in labels]
  
  return L,labels,scores

In [5]:
def remove_notas(bboxs,labels,scores):
  n = len(bboxs)
  L = []
  for i in range(0,n):
    k = len(bboxs[i:])
    bb1 = bboxs[i]
    l1 = labels[i]
    s1 = scores[i]
    for j in range(0,k):
      bb2 = bboxs[i:][j]
      l2 = labels[i:][j]
      s2 = scores[i:][j]
      if bb1[2]-bb1[0] >= 55 and 'nota' in l1:
        L.append(i)
      elif get_iou(bb1, bb2) > 0.5 and 'nota' in l1 and 'nota' in l2:
        if l1 > l2:
          L.append(i+j)
        if l1 < l2:
          L.append(i)
        if l1 == l2 and get_iou(bb1, bb2) < 1:
          if s1 > s2:
            L.append(i+j)
          else:
            L.append(i)
  L = list(np.unique(L))
  b,l,s = [],[],[]
  for i in range(0,n):
    if not(i in L):
      b.append(bboxs[i])
      l.append(labels[i])
      s.append(scores[i])
  return b,l,s

## Modelos

- **Detección**

In [ ]:
%%capture
state_dict = torch.load('mejor.pth')
detector = faster_rcnn.model(num_classes=35)

In [ ]:
detector.load_state_dict(state_dict)

- **Clasificación altura notas**

In [8]:
%%capture
altura_notas = load_learner('export.pkl')

## Predicción
- Transforma la imagen en una secuencia de elementos musicales

In [ ]:
  claves = {'claves':{-4:53, -3:55, -2:57, -1:59, 0:60, 1:62, 2:64, 3:65, 4:67, 5:69, 6:71, 7:72, 8:74, 9:76, 10:77, 11:79, 12:81, 13:83, 14:84},
            'clavef':{-4:33, -3:35, -2:36, -1:38, 0:40, 1:41, 2:43, 3:45, 4:47, 5:48, 6:50, 7:52, 8:53, 9:55, 10:57, 11:59, 12:60, 13:62, 14:64},
#Cambiar clave de do            'claved':{-4:33, -3:35, -2:36, -1:38, 0:40, 1:41, 2:43, 3:45, 4:47, 5:48, 6:50, 7:52, 8:53, 9:55, 10:57, 11:59, 12:60, 13:62, 14:64}}

In [ ]:
def predict(image): # opencv2 array o path
